In [11]:
import pandas as pd
import random
import faker
import uuid
import psycopg2
from psycopg2.extras import register_uuid
from datetime import datetime, date, timedelta, time

In [12]:
name_gen = faker.Faker()
data = pd.read_csv('drug_consumption_transformed.csv')

education_map = {
    'Left School Before 16 years': 0,
    'Left School at 16 years': 1,
    'Left School at 17 years': 2,
    'Left School at 18 years': 3,
    'Some College,No Certificate Or Degree': 4,
    'Professional Certificate/ Diploma': 5,
    'University Degree': 6,
    'Masters Degree': 7,
    'Doctorate Degree': 8,
}

### doctors

In [13]:
doctors = []
for i in range(51):
    gender = random.randint(1, 3) % 2 == 1
    doctors.append(
        {
            'id': uuid.uuid4(),
            'fio': name_gen.name_male() if gender else name_gen.name_female(),
            'birthdate': date.today() - timedelta(days=random.randint(28 * 365, 60 * 365)),
            'gender': gender,
            'room': i
        }
    )
doctors[:5]

[{'id': UUID('2e6cd8f8-a11b-4014-8d0a-e5bb2fffeb3f'),
  'fio': 'Don Carter',
  'birthdate': datetime.date(1981, 11, 13),
  'gender': True,
  'room': 0},
 {'id': UUID('9ad35a58-bb30-4f41-8110-c065ac5e2be9'),
  'fio': 'Paul Harris',
  'birthdate': datetime.date(1994, 4, 4),
  'gender': True,
  'room': 1},
 {'id': UUID('a77a38bb-6140-42fa-b26b-9dfd40ceec18'),
  'fio': 'Brent Reed',
  'birthdate': datetime.date(1986, 12, 12),
  'gender': True,
  'room': 2},
 {'id': UUID('414849d3-a9e1-4ee6-b507-e82a825040cc'),
  'fio': 'James Hernandez',
  'birthdate': datetime.date(1969, 6, 1),
  'gender': True,
  'room': 3},
 {'id': UUID('c929de26-f7a7-46b1-9ff7-345cc74a23e7'),
  'fio': 'Christopher Braun',
  'birthdate': datetime.date(1965, 5, 25),
  'gender': True,
  'room': 4}]

### drugs

In [14]:

drugs = []
drugs_dict = {}
drugs.append({'id': uuid.uuid4(), 'name': 'Alcohol', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Amphet', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Amyl', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Benzos', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Caff', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Cannabis', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Choc', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Coke', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Crack', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Ecstasy', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Heroin', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Ketamine', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Legalh', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'LSD', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Meth', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Mushrooms', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Nicotine', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'Semer', 'description': ''})
drugs.append({'id': uuid.uuid4(), 'name': 'VSA', 'description': ''})
for drug in drugs:
    drugs_dict[drug['name']] = drug['id']
drugs

[{'id': UUID('c89ab7ee-3fb5-4e70-b817-46d035884316'),
  'name': 'Alcohol',
  'description': ''},
 {'id': UUID('37b4e564-6f6b-4de4-9c77-9f54d5537c64'),
  'name': 'Amphet',
  'description': ''},
 {'id': UUID('5b5c4155-b2c0-4b9c-81f0-fd9cdd793db5'),
  'name': 'Amyl',
  'description': ''},
 {'id': UUID('98421806-3dcc-4aac-aff9-fab436d7a3f1'),
  'name': 'Benzos',
  'description': ''},
 {'id': UUID('19acf24a-ac5b-44dd-b41b-d8bc83fcd1a4'),
  'name': 'Caff',
  'description': ''},
 {'id': UUID('7eb9ce6c-a0bb-4fc0-a1ca-df941d6f8a91'),
  'name': 'Cannabis',
  'description': ''},
 {'id': UUID('2a99b252-21a8-4dd5-aa8e-7cc18b1407ff'),
  'name': 'Choc',
  'description': ''},
 {'id': UUID('5617464c-3f09-4e6d-bb6e-409c4a49e715'),
  'name': 'Coke',
  'description': ''},
 {'id': UUID('246f544f-f711-4ca5-92b2-a8cfb9f0ec05'),
  'name': 'Crack',
  'description': ''},
 {'id': UUID('a167d54f-7f07-44bf-b5ca-4ee5995697ac'),
  'name': 'Ecstasy',
  'description': ''},
 {'id': UUID('d1bb5688-0dbf-4b10-a982-d152b33

### patients

In [15]:
patients = []
patients_personalities = []
patients_consumption = []

for idx, row in data.iterrows():
    patient_id = uuid.uuid4()
    patients.append({
        'id': patient_id,
        'fio': name_gen.name_male() if row['Gender'] else name_gen.name_female(),
        'birthdate': date.today() - timedelta(days=random.randint(1, 365) + 365 * row['Age']),
        'gender': row['Gender'],
        'ethnicity': row['Ethnicity'],
        'education': education_map[row['Education']],
    })
    patients_personalities.append({
        'patient_id': patient_id,
        'n_score': row['Nscore'],
        'e_score': row['Escore'],
        'o_score': row['Oscore'],
        'c_score': row['Cscore'],
        'impulsive': row['Impulsive'],
        'sensation': row['SS']
    })
    for drug in drugs:
        if row[drug['name']] != 'CL0':
            patients_consumption.append({
                'patient_id': patient_id,
                'drug_id': drug['id'],
                'usage': row[drug['name']]
            })

patients_consumption[:30]

[{'patient_id': UUID('a4a86a33-586b-4a43-ad49-abdf32ea5bd4'),
  'drug_id': UUID('c89ab7ee-3fb5-4e70-b817-46d035884316'),
  'usage': 'CL5'},
 {'patient_id': UUID('a4a86a33-586b-4a43-ad49-abdf32ea5bd4'),
  'drug_id': UUID('37b4e564-6f6b-4de4-9c77-9f54d5537c64'),
  'usage': 'CL2'},
 {'patient_id': UUID('a4a86a33-586b-4a43-ad49-abdf32ea5bd4'),
  'drug_id': UUID('98421806-3dcc-4aac-aff9-fab436d7a3f1'),
  'usage': 'CL2'},
 {'patient_id': UUID('a4a86a33-586b-4a43-ad49-abdf32ea5bd4'),
  'drug_id': UUID('19acf24a-ac5b-44dd-b41b-d8bc83fcd1a4'),
  'usage': 'CL6'},
 {'patient_id': UUID('a4a86a33-586b-4a43-ad49-abdf32ea5bd4'),
  'drug_id': UUID('2a99b252-21a8-4dd5-aa8e-7cc18b1407ff'),
  'usage': 'CL5'},
 {'patient_id': UUID('a4a86a33-586b-4a43-ad49-abdf32ea5bd4'),
  'drug_id': UUID('d2bc5d44-3a55-4d8e-8e04-a4cc39318709'),
  'usage': 'CL2'},
 {'patient_id': UUID('c3e0e113-153d-4c81-9664-5840804d5250'),
  'drug_id': UUID('c89ab7ee-3fb5-4e70-b817-46d035884316'),
  'usage': 'CL5'},
 {'patient_id': UUID

### patients_doctors_appointments

In [16]:
patients_doctors_appointments = []

start = date.today() - timedelta(days=random.randint(1, 365) + 365 * 9)
end = date.today() + timedelta(days=random.randint(1, 180))

dates = [start + timedelta(days=i) for i in range((end - start).days)]

# по воскресеньям не работает :)
dates = list(filter(lambda d: d.weekday() != 6, dates))

for working_date in dates:
    working_doctors_num = random.randint(5, 20)
    working_doctors = random.sample(doctors, working_doctors_num)
    for doctor in working_doctors:
        # если рабочие часы с 8 до 18, то максимум пациентов у доктора - 20
        patients_num = random.randint(10, 21)
        patients_with_appointment = random.sample(patients, patients_num)
        for i, patient in enumerate(patients_with_appointment):
            # работает с 8 утра, на каждого пациента полчаса
            appointment_datetime = datetime.combine(working_date, time(8, 0)) + timedelta(minutes=30) * i
            # консультация по записи не состоялась, если дата консультации позже чем сейчас, иначе состоялась с 70% вероятностью 
            is_committed = False if appointment_datetime > datetime.now() else  random.randint(1, 11) > 3
            patients_doctors_appointments.append({
                'id': uuid.uuid4(),
                'patient_id': patient['id'],
                'doctor_id': doctor['id'],
                'appointment_datetime': appointment_datetime,
                'is_committed': is_committed
            })
patients_doctors_appointments
     

[{'id': UUID('40f941bc-e30d-4a65-b18e-20312542f712'),
  'patient_id': UUID('25712bdd-3ae9-4560-b8c2-7e80af4e18e8'),
  'doctor_id': UUID('d15d8e83-ae44-4a15-8d81-aebb987fd772'),
  'appointment_datetime': datetime.datetime(2015, 3, 7, 8, 0),
  'is_committed': False},
 {'id': UUID('5d596330-974a-4e21-9aee-43b1da4a7cf7'),
  'patient_id': UUID('1a3a5d8e-9e4a-4a5b-b6fc-48fba94e1012'),
  'doctor_id': UUID('d15d8e83-ae44-4a15-8d81-aebb987fd772'),
  'appointment_datetime': datetime.datetime(2015, 3, 7, 8, 30),
  'is_committed': True},
 {'id': UUID('5ca82c4b-b2df-4a3b-9cfb-ee7942c54a2d'),
  'patient_id': UUID('a87674e1-df20-4235-ab12-ee0995ddd3d7'),
  'doctor_id': UUID('d15d8e83-ae44-4a15-8d81-aebb987fd772'),
  'appointment_datetime': datetime.datetime(2015, 3, 7, 9, 0),
  'is_committed': True},
 {'id': UUID('5ddebac3-1442-4520-b72e-dd8297cd4ddf'),
  'patient_id': UUID('bbaeb614-2c0e-4657-896d-71648d3bf274'),
  'doctor_id': UUID('d15d8e83-ae44-4a15-8d81-aebb987fd772'),
  'appointment_datetime': 

Вставка в БД

In [17]:
conn = psycopg2.connect(
    dbname="test",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
psycopg2.extras.register_uuid(conn)

<psycopg2._psycopg.type 'UUID' at 0x000001D00A8564D0>

In [18]:
cur = conn.cursor()
def insert_all(table_name, rows):
    values = ", ".join(["%s"] * len(rows[0].values()))
    template = f'insert into {table_name} ({", ".join(rows[0].keys())}) values ({values})'
    for row in rows:
        cur.execute(template, tuple(row.values()))

In [19]:
insert_all('clinic.drugs', drugs)
insert_all('clinic.patients', patients)
insert_all('clinic.patients_personalities', patients_personalities)
insert_all('clinic.patients_consumption', patients_consumption)
insert_all('clinic.doctors', doctors)
insert_all('clinic.patients_doctors_appointments', patients_doctors_appointments)
conn.commit()

UniqueViolation: ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "drugs_name_key"
DETAIL:  Ключ "(name)=(Alcohol)" уже существует.


In [20]:
cur.close()